In [1]:
# coming soon

In [2]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import sys
sys.path.append("../../")
import os
import glob
import cv2
import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button, Image, Layout

from components.utils import plotly_helpers as plh

In [3]:
log_directory = os.path.join("..", "logs")
csv_files = glob.glob(os.path.join(log_directory, "ALG_004_*.csv"))
filenames = [os.path.split(csv_file)[-1] for csv_file in csv_files]
opts = [(f,p) for f, p in zip(filenames, csv_files) if "well_known_filters" not in f]

csv_selector = widgets.Dropdown(
    options=opts,
    description='CSV Log file:',
    value='..\\logs\\ALG_004_EXP_002-Baseline-MacLean_et_al-Numba_param_search.csv'
)

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']
metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='Default metric:'
)

In [4]:
VBox([csv_selector, metrics_selector])

In [5]:
selected_file = os.path.abspath(csv_selector.value)

df = plh.load_n_clean(selected_file, gts=False, kernel_sizes=False)
df = df.sort_values(by=metrics_selector.value)

teddy = df[df["scene"]=="teddy"]
cones = df[df["scene"]!="teddy"]

img_widget_teddy = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
img_widget_cones = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))


In [11]:
# Getting Figure widgets and dataframes
fig_teddy, dfs_1 = plh.get_figure_widget_traced(teddy, "match",
                                                metrics_selector.value, "are_occlusions_errors", discrete_hover=True)


fig_cones, dfs_2 = plh.get_figure_widget_traced(cones, "match", 
                                                metrics_selector.value, "are_occlusions_errors", discrete_hover=True)

plh.bind_hover_function_for_traced([fig_teddy], img_widget_teddy, dfs_1)
plh.bind_hover_function_for_traced([fig_cones], img_widget_cones, dfs_2)

dashboard = VBox([HBox([fig_teddy, img_widget_teddy]), 
                 HBox([fig_cones, img_widget_cones])]
                  )


In [12]:
dashboard

    'data': [{'hovertemplate': '%{text}',
              'name': '…